CNN

In [1]:
import keras
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from keras.preprocessing.image import ImageDataGenerator
datagen=ImageDataGenerator(rotation_range=40,width_shift_range=0.2,height_shift_range=0.2,horizontal_flip=True,zoom_range=0.2,rescale=1./255.)
type(datagen)

tensorflow.python.keras.preprocessing.image.ImageDataGenerator

In [19]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Activation
from keras.activations import relu,softmax
from keras.losses import categorical_crossentropy
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler #it will be implemented after every epochs
#the first one will give us value of weights and all 
#to increase and decrease learning rate based on model accuracy
from keras.callbacks import TensorBoard

In [5]:
model=Sequential()

#Single Chunk of CONVOLUTION layer
model.add(Conv2D(32,(3,3),input_shape=(150,150,3),activation='relu'))
#or we can add activation as below
# model.add(Activation='relu')

model.add(MaxPool2D(pool_size=(2,2),strides=(2, 2), padding='valid'))

#Second chunk
model.add(Conv2D(45,(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2),strides=(2, 2), padding='valid'))

#third chunk
model.add(Conv2D(45,(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2),strides=(2, 2), padding='valid'))

model.add(Flatten())
model.add(Dense(1024,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(2,activation='softmax'))



In [16]:
model.compile(loss=keras.losses.binary_crossentropy, optimizer=SGD(), metrics=['accuracy'])

In [14]:
train_gen = datagen.flow_from_directory('data/',target_size=(150,150),#resize the image not crop
                                        batch_size=10)#how many images you want to read per batch
#here we have total 400 we have 40 batches

Found 399 images belonging to 2 classes.


In [24]:
#tb = TensorBoard(log_dir='./logs')#it will make log directory
tb= TensorBoard(log_dir='.') #create folder here only

In [25]:
model_history=model.fit_generator(train_gen,epochs=10 , callbacks=[tb])
#callback will be executed after evry epochs
#callbacks=[TensorBoardColabCallback(tbcolab),loss_history,lrate_]
#we have 400 images and 20 per batch so 400/10 we get 40 steps per epoch

Epoch 1/10
40/40 [==============================] - 13s 311ms/step - loss: 0.6180 - accuracy: 0.6642
Epoch 2/10
40/40 [==============================] - 11s 285ms/step - loss: 0.6130 - accuracy: 0.6667
Epoch 3/10
40/40 [==============================] - 12s 294ms/step - loss: 0.6100 - accuracy: 0.6617
Epoch 4/10
40/40 [==============================] - 12s 286ms/step - loss: 0.6074 - accuracy: 0.6892
Epoch 5/10
40/40 [==============================] - 12s 290ms/step - loss: 0.5974 - accuracy: 0.6717
Epoch 6/10
40/40 [==============================] - 12s 289ms/step - loss: 0.6089 - accuracy: 0.6817
Epoch 7/10
40/40 [==============================] - 12s 288ms/step - loss: 0.5961 - accuracy: 0.6742
Epoch 8/10
40/40 [==============================] - 12s 293ms/step - loss: 0.6028 - accuracy: 0.6667
Epoch 9/10
40/40 [==============================] - 12s 289ms/step - loss: 0.5882 - accuracy: 0.6942
Epoch 10/10
40/40 [==============================] - 12s 289ms/step - loss: 0.6015 - accura

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 72, 72, 45)        13005     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 36, 36, 45)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 34, 34, 45)        18270     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 17, 17, 45)        0         
_________________________________________________________________
flatten (Flatten)            (None, 13005)            

In [ ]:
#learning rate schedular
def step_decay(epoch):
    initial_rate=0.1
    drop=0.5
    epoch_drop=3.0
    lrate=initial_rate * math.pow(drop,math.floor((1+epoch)/epoch_drop))
    return lrate
lrate_=LearningRateScheduler(step_decay)